In [5]:
import sys, csv, os
import numpy as np
import pandas as pd
import pdb

In [6]:
paths = {'trial1':'',
         'trial2':r"C:\Users\agutf\Sync\repos\apk-development\analysis\vaccine_cox\draft2\events.csv",
         'trial3':r"C:\Users\agutf\Sync\repos\apk-development\analysis\vaccine_cox\candidates\simphony_model_1604889008444_64pc\instance_1\output\events.csv",
         'trial4':r"C:\Users\agutf\Sync\repos\apk-development\analysis\vaccine_cox\candidates\simphony_model_1604882689569_35pc\instance_1\output\events.csv"}
#toy trial
#events_fpath = r"C:\Users\agutf\AppData\Local\Temp\simphony_model_1593922625316\instance_1\output\events.csv"
chosen_trial = 'trial4'
#D2E60
events_fpath = paths[chosen_trial]

In [7]:
time_between_bleeds = 7
reporting_frequency = time_between_bleeds/365
#rebase_time_to_final_dose

In [8]:
#events_file = open(events_file,'r')
#events = csv.DictReader(events_file)

In [9]:
#events_file.close()

In [10]:
trial_header = pd.read_csv(events_fpath, nrows=2, header=1)

In [11]:
trial_header

,CNEP_data_source,CNEP_plus_file,NEP_data,NHBS_data,ab_prob_acute,ab_prob_chronic,attrition_rate,burn_in_days,dump_network_distances,homophily_strength,...,vaccine_enrollment_probability_positiveinnetwork,vaccine_enrollment_probability_unbiased,vaccine_followup1_periods,vaccine_followup2_periods,vaccine_followup_purge_with_rna,vaccine_followup_weeks,vaccine_schedule,vaccine_study_arm_n,verbosity,Unnamed: 69
0,File,data/cnep_plus_all_2018.02.13.csv,data/NEP_Risk_Base_Merge.csv,data/IDU2_HCV_model_012913_cleaned.csv,0.02,0.67,0.024,365.0,-1.0,0.7,...,1.0,0.0,18.0,9.0,0.0,4.0,D2E60,1599.0,populations+events,NaN
1,NOTE: we don't record failed exposure events o...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
burn_in_days = trial_header.burn_in_days[0]
vaccine_schedule = trial_header.vaccine_schedule[0]
vaccine_enrollment_launch_day = trial_header.vaccine_enrollment_launch_day
vaccine_trial_end_date = 1400/365.0 #external default

In [13]:
#vaccine_trial_end_date = 2*vaccine_trial_end_date

In [14]:
vaccine_schedule

'D2E60'

In [15]:
events = pd.read_csv(events_fpath, header=4)
events.rename(columns={'Num Buddies':'total_network_size'}, inplace=True)

In [16]:
events.tail()

,Time,EC,Agent,Event,Info,L1,N1,L2,N2,C,...,current_total_network_size,Daily_injection_intensity,Fraction_recept_sharing,DBLabel,trial_arm,chicago_community_name,fraction_total_network_female,fraction_total_network_nh_white,fraction_total_network_inder30,mean_age_total_network
120608,9.858,NaN,395604532,deactivated,NaN,-,-,-,-,NaN,...,0,1.333333,0.000000,537,noarm,OutsideChicago,NaN,NaN,NaN,NaN
120609,9.858,NaN,7549754,deactivated,NaN,-,-,-,-,NaN,...,0,8.000000,0.000000,2919,noarm,OutsideChicago,NaN,NaN,NaN,NaN
120610,9.858,NaN,1694354633,infected,NaN,Time_to_exposure,0.5726027397260274,-,-,NaN,...,2,3.130998,0.727596,5936,study,OutsideChicago,0.0,0.5,0.0,45.828767
120611,9.858,NaN,536181101,activated,NaN,-,-,-,-,NaN,...,0,3.000000,0.000000,4417,noarm,OutsideChicago,NaN,NaN,NaN,NaN
120612,9.858,NaN,1440828996,activated,NaN,-,-,-,-,NaN,...,0,0.533333,0.000000,2948,noarm,OutsideChicago,NaN,NaN,NaN,NaN


In [17]:
events[events.Event == 'enteredfollowup'].head()

,Time,EC,Agent,Event,Info,L1,N1,L2,N2,C,...,current_total_network_size,Daily_injection_intensity,Fraction_recept_sharing,DBLabel,trial_arm,chicago_community_name,fraction_total_network_female,fraction_total_network_nh_white,fraction_total_network_inder30,mean_age_total_network
34022,1.247,NaN,770432138,enteredfollowup,RNA=false;arm=study,-,-,-,-,NaN,...,2,0.236102,0.049801,7408,study,OutsideChicago,0.500000,1.000000,0.000000,36.206849
34024,1.247,NaN,1007672333,enteredfollowup,RNA=false;arm=placebo,-,-,-,-,NaN,...,12,5.980163,0.239044,10892,placebo,WestSide,0.333333,0.333333,0.166667,37.194521
34026,1.247,NaN,273570154,enteredfollowup,RNA=false;arm=study,-,-,-,-,NaN,...,2,4.196250,0.436247,11620,study,OutsideChicago,1.000000,0.500000,0.000000,37.050685
34028,1.247,NaN,2002807992,enteredfollowup,RNA=false;arm=placebo,-,-,-,-,NaN,...,1,3.539556,0.471181,11781,placebo,OutsideChicago,1.000000,1.000000,1.000000,29.093151
34030,1.247,NaN,549480026,enteredfollowup,RNA=false;arm=study,-,-,-,-,NaN,...,1,1.758938,0.635202,9577,study,WestSide,1.000000,0.000000,0.000000,43.876712


In [18]:
pd.unique(events.HCV)

array(['susceptible', 'chronic', 'recovered', 'infectiousacute',
       'exposed', 'vaccinated'], dtype=object)

In [19]:
trial_starts = events[events.Event == 'trialstarted'].copy()
trial_starts['RNA_at_start_of_trial'] = trial_starts.Info.str.contains('RNA=true')
trial_starts = trial_starts.drop(columns=['EC','L1','L2','N1','N2','C','Info','Event','DBLabel','HCV'])
trial_starts = trial_starts.rename(columns={'Time':'time_entered_trial'})

agent_baselines = {}
for row in trial_starts.iterrows():
    agent_data = row[1]
    agent = agent_data.Agent
    agent_baselines[agent] = agent_data

In [20]:
agent_data

time_entered_trial                     2.082
Agent                              709613653
Age                                  40.0356
Gender                                  Male
Race                                Hispanic
Zip                                    60639
Syringe_source                            HR
HCV_friend_preval                          0
Age_Started                               19
Drug_in_degree                             1
Drug_out_degree                            0
current_total_network_size                 1
Daily_injection_intensity                  9
Fraction_recept_sharing                    0
trial_arm                              study
chicago_community_name              WestSide
fraction_total_network_female              0
fraction_total_network_nh_white            0
fraction_total_network_inder30             1
mean_age_total_network               26.0822
RNA_at_start_of_trial                  False
Name: 59480, dtype: object

In [21]:
#trial_starts[trial_starts.Agent==254477147]

In [22]:
events2 = events[events.Agent.isin(agent_baselines)]

In [23]:
vaccinations = events2[(events2.Time <= vaccine_trial_end_date ) & (events2.Event == 'vaccinated')]
agent_vaccinations = vaccinations.groupby(['Agent']).size().to_dict()

In [24]:
ever_infected = events2[(events2.Time <= vaccine_trial_end_date ) & (events2.Event == 'infected')]
infected_count = ever_infected.groupby(['Agent']).size().to_dict()
#for agents with infections, count of the number of infections

In [25]:
#ever_recovered = events2[(events2.Time <= vaccine_trial_end_date ) & (events2.Event == 'recovered')]
#recovered_count = ever_recovered.groupby(['Agent']).size().to_dict()

In [26]:
#events2[events2.Agent==254477147]

In [27]:
#events2[events2.Agent == list(agent_baselines.keys())[2]]

In [28]:
pd.unique(events2.Event)

array(['activated', 'trialstarted', 'vaccinated', 'trialabandoned',
       'deactivated', 'infected', 'enteredfollowup', 'infectious',
       'infollowup', 'chronic', 'infollowup2', 'recovered',
       'trialcompleted'], dtype=object)

In [29]:
#events2[events2.Agent==293654963]

In [30]:
#the following events do not provide new information about the RNA state or the censorship
# if the agent becomes infected or recovered, we will retrieve it from "infected" and "recovered" events
#  remove to simplify the parsing
# added infectious and chronic for safety's sake
events2 = events2[~events2.Event.isin
                  (['activated','enteredfollowup','infollowup','infollowup2','vaccinated'])]

In [31]:
pd.unique(events2.Event)

array(['trialstarted', 'trialabandoned', 'deactivated', 'infected',
       'infectious', 'chronic', 'recovered', 'trialcompleted'],
      dtype=object)

In [32]:
pd.unique(events2.HCV)

array(['vaccinated', 'exposed', 'infectiousacute', 'chronic', 'recovered'],
      dtype=object)

In [33]:
agent_events = {}
longest_history = 0 #needed for padding the data
for agent in agent_baselines:
    a_events = events2[events2.Agent == agent][['Event','HCV','Time']]
    a_events = a_events[a_events.Time <= vaccine_trial_end_date]
    a_events = a_events.sort_values(by="Time") #just in case
    a_events = a_events.append(pd.DataFrame([['fieldworkcompleted', '-', vaccine_trial_end_date]], columns=list(['Event','HCV', 'Time'])))
    a_events["StepsFromEnrollmentStart"] = (a_events.Time - agent_baselines[agent]["time_entered_trial"])/reporting_frequency
    agent_events[agent] = a_events
    longest_history = max(longest_history, a_events.Time.iloc[-1]-a_events.Time.iloc[0])  
                          #min(vaccine_trial_end_date-a_events.Time.iloc[0], 
                          #    a_events.Time.iloc[-1]-a_events.Time.iloc[0]))


In [34]:
#a_events

In [35]:
print(longest_history)
longest_history_len = longest_history/reporting_frequency
print(longest_history_len)
#for agent, a_events in agent_events.items():
#    enrollment_start = agent_events[agent].Time.iloc[0]
#    if a_events['Time'].iloc[-1] < longest_history:
#        agent_events[agent] = agent_events[agent].append(pd.DataFrame([['padding', 'padding', longest_history]], columns=list(['Event','HCV', 'Time'])))
#    agent_events[agent].sort_values(by="Time", inplace=True)

2.753616438356164
143.58142857142855


In [36]:
#agent=list(agent_events.keys())[1]

In [37]:
#agent_events[agent]

In [38]:
pd.unique(events2.Event)

array(['trialstarted', 'trialabandoned', 'deactivated', 'infected',
       'infectious', 'chronic', 'recovered', 'trialcompleted'],
      dtype=object)

In [39]:
pd.unique(events2.HCV)

array(['vaccinated', 'exposed', 'infectiousacute', 'chronic', 'recovered'],
      dtype=object)

In [40]:
'''
produce a sequence [+,-,-,...,lost,lost,...,padding,padding,..]
'''
def estimate_history(my_events, RNA_positive, target_length):
    current_hcv = 'RNA_positive' if RNA_positive else 'RNA_negative'
    last_hcv_reading = current_hcv #last known hcv status
    in_tracking = True
    #pdb.set_trace()
    h = [] #test the correct count [current_hcv]
    assert my_events.shape[0]>1
    for i, new_event in enumerate(my_events.Event):
        if i == 0:
            continue
        full_steps_to_new = int(my_events.StepsFromEnrollmentStart.iloc[i] - \
                                my_events.StepsFromEnrollmentStart.iloc[i-1])
        h = h + [current_hcv] * full_steps_to_new
        
        #full_steps_to_new might be less than 1, in which case we will add nothing, but the state will change
        if new_event in frozenset(['infected', 'infectious', 'chronic']) and in_tracking:
            current_hcv = 'RNA_positive'
            last_hcv_reading = current_hcv
        elif new_event == 'recovered' and in_tracking:
            current_hcv = 'RNA_negative'
            last_hcv_reading = current_hcv
        elif new_event in frozenset(['trialabandoned', 'deactivated']):
            current_hcv = 'lost_to_followup'
            in_tracking = False
        elif new_event in frozenset(['trialcompleted', 'fieldworkcompleted']): 
            break
        elif not in_tracking:
            continue
        else:
            print(new_event)
            pdb.set_trace()

            assert False
        #invariant: h contains all the records up to my_events.StepsFrom[i]. current_hcv is the new state
    
    h = h + ['padding']*int(target_length-len(h))
    
    #pdb.set_trace()
    #print(len(h))
    return h, last_hcv_reading

In [41]:
#estimate_history(list(agent_events.values())[2], 
#                 agent_baselines[agent]['RNA_at_start_of_trial'], 
#                 longest_history_len)

In [42]:
#list(agent_events.values())[2]

In [43]:
agent_histories = {}
for agent, a_events in agent_events.items():
    agent_baselines[agent]['doses_received'] = agent_vaccinations[agent]
    
    regular_history, last_hcv_reading = estimate_history(a_events, 
                                                 agent_baselines[agent]['RNA_at_start_of_trial'],
                                                 longest_history_len)

    agent_baselines[agent]['nonchronic_abpos_at_final_reading']   = \
               (agent_baselines[agent]['trial_arm'] == 'study' and last_hcv_reading == 'RNA_negative') or \
               (agent_baselines[agent]['trial_arm'] == 'placebo' and (agent in infected_count) and last_hcv_reading == 'RNA_negative')
    #print(agent_baselines[agent]['nonchronic_abpos_at_final_reading'])
    #print(agent_baselines[agent]['trial_arm'])
    #print(agent in infected_count)
    #print(last_hcv_reading)
    
    agent_histories[agent] = agent_baselines[agent].append(pd.Series(regular_history))

In [44]:
#agent_histories[agent]

In [45]:
a_events

,Event,HCV,Time,StepsFromEnrollmentStart
59480,trialstarted,vaccinated,2.082000,0.000000
101743,trialcompleted,vaccinated,3.625000,80.456429
0,fieldworkcompleted,-,3.835616,91.438571


In [46]:
agent_histories[agent]

time_entered_trial        2.082
Agent                 709613653
Age                     40.0356
Gender                     Male
Race                   Hispanic
                        ...    
138                     padding
139                     padding
140                     padding
141                     padding
142                     padding
Length: 166, dtype: object

In [47]:
vaccine_trial_end_date

3.835616438356164

In [48]:
output_df = pd.DataFrame.from_dict(agent_histories, orient='columns')

#agent_0 = agent_histories.keys()[0]
#output_df.set_index(agent_baselines[agent_0].keys() + range(longest_history), inplace=True)

In [49]:
output_df

,770432138,1007672333,273570154,2002807992,549480026,35088813,289901099,532537952,1479258741,1726394741,...,684095337,65364501,414380720,815188323,1078420130,1301635411,347463549,1678248644,275803261,709613653
time_entered_trial,1.082,1.082,1.082,1.082,1.082,1.082,1.082,1.082,1.082,1.085,...,2.079,2.082,2.082,2.082,2.082,2.082,2.082,2.082,2.082,2.082
Agent,770432138,1007672333,273570154,2002807992,549480026,35088813,289901099,532537952,1479258741,1726394741,...,684095337,65364501,414380720,815188323,1078420130,1301635411,347463549,1678248644,275803261,709613653
Age,34.9507,36.0658,44.5863,38.274,44.6603,26.0712,18.0658,23.1014,26.6438,35.4877,...,35.874,26.1753,27.0767,24.337,21.1068,19.0521,35.0685,31.7178,20.0603,40.0356
Gender,Male,Female,Male,Female,Female,Male,Male,Female,Male,Male,...,Female,Male,Male,Male,Male,Female,Male,Male,Male,Male
Race,Hispanic,NHBlack,NHWhite,NHBlack,NHBlack,NHWhite,NHWhite,NHWhite,NHWhite,Hispanic,...,NHBlack,NHWhite,NHBlack,NHWhite,NHWhite,NHWhite,NHWhite,NHWhite,NHWhite,Hispanic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,lost_to_followup,padding,padding,padding,padding,padding,padding,padding,padding,lost_to_followup,...,padding,padding,padding,padding,padding,padding,padding,padding,padding,padding
139,lost_to_followup,padding,padding,padding,padding,padding,padding,padding,padding,lost_to_followup,...,padding,padding,padding,padding,padding,padding,padding,padding,padding,padding
140,lost_to_followup,padding,padding,padding,padding,padding,padding,padding,padding,lost_to_followup,...,padding,padding,padding,padding,padding,padding,padding,padding,padding,padding
141,lost_to_followup,padding,padding,padding,padding,padding,padding,padding,padding,lost_to_followup,...,padding,padding,padding,padding,padding,padding,padding,padding,padding,padding


In [52]:
fp = f'{events_fpath}_history_{chosen_trial}_gap{time_between_bleeds}.csv'
fp

'C:\\Users\\agutf\\Sync\\repos\\apk-development\\analysis\\vaccine_cox\\candidates\\simphony_model_1604882689569_35pc\\instance_1\\output\\events.csv_history_trial4_gap7.csv'

In [50]:
output_df.to_csv(fp)

In [47]:
os.getcwd()

'C:\\Users\\agutf\\Sync\\repos\\apk-development\\analysis'

-----